In [2]:
!pip install simpletransformers

     ------------------------------------ 250.5/250.5 kB 451.5 kB/s eta 0:00:00
     --------------------------------------- 43.6/43.6 kB 47.5 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 1.1/1.1 MB 2.1 MB/s eta 0:00:00
     -------------------------------------- 452.9/452.9 kB 1.3 MB/s eta 0:00:00
     ---------------------------------------- 1.9/1.9 MB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 9.2/9.2 MB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 1.3 MB/s eta 0:00:00
     ---------------------------------------- 5.8/5.8 MB 2.5 MB/s eta 0:00:00
     -------------------------------------- 182.4/182.4 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished wit

In [3]:
import numpy as np
import pandas as pd
import simpletransformers
import re
from simpletransformers.classification import ClassificationModel
import itertools

In [4]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [5]:
df=pd.read_csv('headlines.csv')

In [6]:
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [5]:
df.shape

(26709, 3)

In [6]:
df.isnull().sum()

article_link    0
headline        0
is_sarcastic    0
dtype: int64

# Cleaning Text

In [ ]:
def clean_text(a):
  x=re.sub('[^a-zA-Z]',' ',a)
  return x

In [ ]:
df['headline']=df['headline'].apply(clean_text)

# Preparing Data for transformer input

In [ ]:
headlines=df['headline']

In [10]:
headlines

0        former versace store clerk sues over secret  b...
1        the  roseanne  revival catches up to our thorn...
2        mom starting to fear son s web series closest ...
3        boehner just wants wife to listen  not come up...
4        j k  rowling wishes snape happy birthday in th...
                               ...                        
26704                 american politics in moral free fall
26705                              america s best    hikes
26706                                reparations and obama
26707    israeli ban targeting boycott supporters raise...
26708                    gourmet gifts for the foodie     
Name: headline, Length: 26709, dtype: object

In [11]:
target=df['is_sarcastic']
target

0        0
1        0
2        1
3        1
4        0
        ..
26704    0
26705    0
26706    0
26707    0
26708    0
Name: is_sarcastic, Length: 26709, dtype: int64

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(headlines,target,test_size=0.3,random_state=42)

In [ ]:
train_df = pd.concat([x_train, y_train], axis=1)

In [ ]:
test_df = pd.concat([x_test, y_test], axis=1)

In [15]:
train_df.head()

,headline,is_sarcastic
5874,teen study bible found to increase fun of reli...,1
15611,montana gop candidate owns stake in company ac...,0
5303,u s best ranked cities for hotels,0
2703,starting a small business is anything but routine,0
6586,domestic terrorists organizing online are rea...,0


In [16]:
test_df.head()

,headline,is_sarcastic
6913,isis recruiter excited to be talking to popula...,1
11611,jimmy fallon could barely keep it together dur...,0
25947,drunk ron weasley wishing harry potter happy ...,0
24478,hoosier hostility not the american way,0
11863,turns out running doesn t wreck your knees aft...,0


# Transformer Model

In [ ]:
train_args = {
    'evaluate_during_training': True,
    'logging_steps': 100,
    'num_train_epochs':2,
    'evaluate_during_training_steps': 100,
    'save_eval_checkpoints': False,
    'train_batch_size': 32,
    'eval_batch_size': 32,
    'overwrite_output_dir': True,
    'fp16': False
}

In [ ]:
model_BERT = ClassificationModel('bert', 'bert-base-cased', num_labels=2, use_cuda=True,args=train_args)

In [19]:
model_BERT.train_model(train_df, eval_df=test_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:251: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Running loss: 0.755604

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


Running loss: 0.403868

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:669: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running loss: 0.065683


Running loss: 0.052263



In [20]:
result, model_outputs, wrong_predictions = model_BERT.eval_model(test_df, acc=sklearn.metrics.accuracy_score)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:669: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


In [21]:
result

{'acc': 0.9105204043429427,
 'eval_loss': 0.30793343793111017,
 'fn': 362,
 'fp': 355,
 'mcc': 0.8188474300815678,
 'tn': 4089,
 'tp': 3207}

## Predicting Some New Values

In [22]:
predictions,raw_outputs=model_BERT.predict(['some guy who s not stephen colbert to deliver college commencement speech'])

In [23]:
predictions

array([1])